# Getting started with BioNumPy

concepts:

1. Using NumPy on BioNumPy datasets
2. Reading datasets and performing operations on dataset chunks
3. Combining analysis results from chunks of data


## Install and import

BioNumPy can easily be installed through pip:

In [1]:
!pip install bionumpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00


Testing that the installation worked by importing BioNumPy and encode a sequence:

In [2]:
import numpy as np
import bionumpy as bnp
sequence = bnp.as_encoded_array("ACTG")
print(sequence)

ACTG


## Part 1: Introduction to BioNumPy datasets

BioNumPy datasets can consist of things like DNA sequences, sequence names, base qualities, proteins sequences, etc.
They are usually created by reading files (e.g. fastq, vcfs etc), but we can also create small datasets using the
`bnp.as_encoded_array()` function:

In [3]:
sequences = bnp.as_encoded_array([
    "ACTGACG",
    "ACA",
    "ACACGGAAC"
])
print(sequences)

ACTGACG
ACA
ACACGGAAC


The `sequences` object is encoded and represented using an efficient NumPy-like data structure. However, BioNumPy doesn't require you to know about any of internals or details, and we can just treat the data as NumPy-matrix consisting of DNA and not numbers. For instance, getting a boolean mask with the positions of Gs is as simple as:

In [4]:
is_g = sequences == "G"
print(is_g)

[False False False  True False False  True]
[False False False]
[False False False False  True  True False False False]


We can then take the `np.sum` of this mask to count the number of Gs:

In [5]:
np.sum(is_g)

4



What do you think happens if you specify `axis=1` on np.sum()? What does the output tell you? Try running the code below and see if you can make sense of the output:

In [7]:
np.sum(is_g, axis=1)

array([2, 0, 2])




1. Make a mask with the positions of C
2. Count how many bases are either C or G
3. Compute the GC-content (PS: You might get use of `np.mean`)
4. Make a new set of sequences where the first base pairs are removed

Solution

In [6]:
is_c = sequences == "C"
is_c_or_g = is_c | is_g  # the | operator is "or"
# number of c or g:
np.sum(is_c_or_g)

print("GC content:")
gc_content = np.mean(is_c_or_g)  # alternatively sum and divide by number of bases
print(gc_content)

print("Stripped sequences:")
stripped_sequences = sequences[:, 1:]  # we index on last dimension (columns)
print(stripped_sequences)


GC content:
0.5263157894736842
Stripped sequences:
CTGACG
CA
CACGGAAC


## Part 2: Working with files

In the previous task, we worked with a very small dataset consisting of only three sequences. When working with larger datasets, we want to avoid reading the whole data into memory. Instead, BioNumPy reads chunks of data, and we typically analyse each chunk seperately and combine the results in the end if necessary.

In this and the coming exercises, we will work with ChIP-seq data. We start by downloading FASTQ reads for a CTCF ChIP-seq experiment from the Encode Project:

In [1]:
!wget https://www.encodeproject.org/files/ENCFF000RWH/@@download/ENCFF000RWH.fastq.gz

'wget' is not recognized as an internal or external command,
operable program or batch file.


After running the command above, you will get a file `ENCFF000RWH.fastq.gz`. You can open and read a chunk from the file with BioNumPy like this:

In [9]:
f = bnp.open("ENCFF000RWH.fastq.gz")
chunk = f.read_chunk()
print(chunk)

SequenceEntryWithQuality with 44730 entries
                     name                 sequence                  quality
  HWI-EAS306_2_FC307KC_CR  GGAGAGCAAGCATTCCACTACCT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAAGAAAGGATATCAGTGATTGA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTTGATAAACATTCAGAGAATG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTGATAGATGTGCTAACTAGTCT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTGCTAAATGGATAGCTCAAATA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTAACACCCTGGCTTATCTGAG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAATGGAATGGAATGGAATGGAA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTAGATAAATTGTTCTATGCAT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAATTGGGCTGTGGTTTTCCTTG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAGTGATGATAGTTCATGGTGAG  [40 40 40 40 40 40 29 4


`chunk` is now an object containing a part of the file. BioNumPy automatically detected that this is a FASTQ file, and chooses a suitable data structure. We can access the sequences, names and qualities from this data structure:

In [10]:
print(chunk.sequence)
print(chunk.name)
print(chunk.quality)

GGAGAGCAAGCATTCCACTACCTATGGAGGACAAA
GAAGAAAGGATATCAGTGATTGAAGATCAAATTAA
GTTTGATAAACATTCAGAGAATGGTATTAGATTTT
GTGATAGATGTGCTAACTAGTCTGATTTGATCACC
GTGCTAAATGGATAGCTCAAATACCCGGTCTCTTG
GTTAACACCCTGGCTTATCTGAGACATAAATAATG
GAATGGAATGGAATGGAATGGAATGGAATGGAATG
GTTAGATAAATTGTTCTATGCATCATCTGTAATGA
GAATTGGGCTGTGGTTTTCCTTGTTGTTGATTACT
GAGTGATGATAGTTCATGGTGAGATAATTTCCAAG
GATTCTGCTGTTTTCTTAAATAAATGTTTCCCAGA
GAGGTATCCATCACTTTAAGCCTTATTCTTTCTTT
GAAGACAATACCTATTAACACCACTTTCTTTGCCC
GAGGAACTTACAGAAAGATGGAAGAAGAGTATGCC
GTGTTTCAAAACTGCTCTATCAAGAGAAATGGTCC
GGAAAACTGGCTAGCCATATGTAGAAAGCTGAAAC
GGGGTTTATATAGAACGTTCTAGGATTCTAGTTCA
GAACAGTACTATTTACCTTGGGCTATCAAAGATCA
GAATATTCTAGATGGCATAAGCATAGAGTGCCCAA
GTGCCATCTGATTAGAGGACAAGAGATGAAAGGAA
HWI-EAS306_2_FC307KC_CR_7_1_727_1926
HWI-EAS306_2_FC307KC_CR_7_1_763_1926
HWI-EAS306_2_FC307KC_CR_7_1_826_1970
HWI-EAS306_2_FC307KC_CR_7_1_219_389
HWI-EAS306_2_FC307KC_CR_7_1_207_382
[40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 25 40 40 32 22
 27 28 27 38 40 29 18 17

These objects work similarily to the `sequences` object in the previous task, meaning that NumPy-functions are compatible with them. We can also index the chunk like a NumPy array. For instance, getting the first three reads can be done with:

In [11]:
print(chunk[0:3])

SequenceEntryWithQuality with 3 entries
                     name                 sequence                  quality
  HWI-EAS306_2_FC307KC_CR  GGAGAGCAAGCATTCCACTACCT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAAGAAAGGATATCAGTGATTGA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTTGATAAACATTCAGAGAATG  [40 40 40 40 40 40 40 4


... and similarily, getting the first three bases of each read is as simple as:

In [12]:
print(chunk.sequence[:, 0:3])

GGA
GAA
GTT
GTG
GTG
GTT
GAA
GTT
GAA
GAG
GAT
GAG
GAA
GAG
GTG
GGA
GGG
GAA
GAA
GTG




* Computing the average base quality value of all the reads in the chunk
* Computing the average base quality without considering the first 5 and last 5 base pairs of all the reads

In [17]:
import numpy as np
import bionumpy as bnp
f = bnp.open("ENCFF000RWH.fastq.gz")
# Reads a chunk from the file
chunk = f.read_chunk()

Computes the average base quality value of all the reads in the chunk




In [18]:
average_quality = chunk.quality.mean()
print("Average base quality of all reads in the chunk:", average_quality)

Average base quality of all reads in the chunk: 29.40168503081984


* Compute the average base quality without considering the first 5 and last 5 base pairs of all the reads

* Remove the first 5 and last 5 base pairs from each read

* Compute the average quality score without the first 5 and last 5 base pairs

In [20]:
trimmed_quality = chunk.quality[:, 5:-5]
average_trimmed_quality = trimmed_quality.mean()
print("Average base quality without the first 5 and last 5 base pairs of all reads:", average_trimmed_quality)

Average base quality without the first 5 and last 5 base pairs of all reads: 29.91046098815113


 NumPy-functions such as `np.mean` and `np.sum` can take an axis-argument. `axis=0` performs the operation over the first axis (e.g. computes one number for every base if we have reads) wile `axis=1` performs the operation over the rows (e.g. computes one number for every read)

* Finding the average base quality for **each read** in this chunk (axis=1)
* How many reads have average base quality lower than 20?
* Subset the chunk so that we are left with reads with average base qualities >= 20.
* How many reads are there in your new filtered chunk?
* Putting  code for filtering the reads in a function. The function should take a chunk as an argument and return a new "filtered" chunk.

In [13]:
def filter_reads(chunk):
    mask = np.mean(chunk.quality, axis=1) >= 20
    return chunk[mask]

f = bnp.open("ENCFF000RWH.fastq.gz")
chunk = f.read_chunk()
print(chunk)

filtered_chunk = filter_reads(chunk)
print(filtered_chunk)

SequenceEntryWithQuality with 44730 entries
                     name                 sequence                  quality
  HWI-EAS306_2_FC307KC_CR  GGAGAGCAAGCATTCCACTACCT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAAGAAAGGATATCAGTGATTGA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTTGATAAACATTCAGAGAATG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTGATAGATGTGCTAACTAGTCT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTGCTAAATGGATAGCTCAAATA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTAACACCCTGGCTTATCTGAG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAATGGAATGGAATGGAATGGAA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTAGATAAATTGTTCTATGCAT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAATTGGGCTGTGGTTTTCCTTG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAGTGATGATAGTTCATGGTGAG  [40 40 40 40 40 40 29 4
SequenceEntryWithQuality with 41979 entries
                     name                 sequence                  quality


## Part 3: Working with chunks from files
Above,  code is  for filtering a chunk of sequences. This allows us to read chunks iteratively from a file, and run our function on each chunk. Working on chunks like this  keep memory usage low while still getting significant speedup from NumPy (as compared to working on single reads, which is common when writing vanilla Python programs).

Below, we read chunks iteratively using the `file.read_chunks()` method, filter each chunk and write the resulting filtered chunks to a new file:

In [14]:
f = bnp.open("ENCFF000RWH.fastq.gz")
out_file = bnp.open("ENCFF000RWH_filtered.fastq", "w")

for chunk in f.read_chunks():
    print(chunk)
    filtered_chunk = filter_reads(chunk)
    print(filtered_chunk)
    out_file.write(filtered_chunk)

SequenceEntryWithQuality with 44730 entries
                     name                 sequence                  quality
  HWI-EAS306_2_FC307KC_CR  GGAGAGCAAGCATTCCACTACCT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAAGAAAGGATATCAGTGATTGA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTTGATAAACATTCAGAGAATG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTGATAGATGTGCTAACTAGTCT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTGCTAAATGGATAGCTCAAATA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTAACACCCTGGCTTATCTGAG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAATGGAATGGAATGGAATGGAA  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GTTAGATAAATTGTTCTATGCAT  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAATTGGGCTGTGGTTTTCCTTG  [40 40 40 40 40 40 40 4
  HWI-EAS306_2_FC307KC_CR  GAGTGATGATAGTTCATGGTGAG  [40 40 40 40 40 40 29 4
SequenceEntryWithQuality with 41979 entries
                     name                 sequence                  quality



## Part 4: Combining analysis results from chunks

A common pattern when working with big datasets is to perform an analysis on parts of the dataset (chunks) and combine the results.

For instance, if we want to compute the average base quality for the whole data set, but we don't want to read the whole data set into memory.

BioNumPy lets us do computation on single chunks, and provides utility functions for merging the results. This is done by adding the `bnp.streamable()` decorator.

For instance, here we have defined a function that computes the number of matches of the subsequence CCCTC in **a single chunk**:

In [21]:
from bionumpy.sequence.string_matcher import match_string
def count_reads_with_matches(chunk):
    # Makes a boolean mask with True where we have a match and False elsewhere
    matches = match_string(chunk.sequence, "CCCTC")
    return np.sum(matches)

If we then want the number of matches for our whole read dataset, we could call the function per chunk like this:

In [22]:
f = bnp.open("ENCFF000RWH_filtered.fastq")
results = []
for chunk in f.read_chunks():
    results.append(count_reads_with_matches(chunk))

print(sum(results))

368126


To avoid doing the for-loop above, BioNumPy provides a decorator `@streamable()`
that can be added above a function in order to make the function able to handle multiple chunks. BioNumPy will automatically
run the function on each chunk and combine the results using the function provided with the decorator, in this case `sum`:

In [23]:
@bnp.streamable(sum)
def count_reads_with_matches(chunk):
    matches = match_string(chunk.sequence, "CCCTC")
    return np.sum(matches)

chunks = bnp.open("ENCFF000RWH_filtered.fastq").read_chunks()
print(count_reads_with_matches(chunks))

368126


**TASK**

* Write a function for counting the total number of Cs in a chunk
* Test your function on one chunk
* Add the streamable-decorator with `sum` (as above) and compute the total number of Cs in the whole fastq data set.

In [24]:
import bionumpy as bnp
import numpy as np

# Function to count the total number of 'C's in a chunk
def count_cs(chunk):
    # Count occurrences of 'C' in each sequence
    return np.sum(chunk.sequence == 'C')

# Test the function on one chunk
f = bnp.open("ENCFF000RWH.fastq.gz")
chunk = f.read_chunk()
print("Total number of 'C's in the chunk:", count_cs(chunk))

Total number of 'C's in the chunk: 348145


In [25]:
# Adding the streamable decorator with sum to compute the total number of 'C's in the whole FASTQ dataset
@bnp.streamable(sum)
def count_cs(chunk):
    # Count occurrences of 'C' in each sequence
    return np.sum(chunk.sequence == 'C')

In [26]:
#Computes the total number of 'C's in the whole FASTQ dataset
chunks = bnp.open("ENCFF000RWH.fastq.gz").read_chunks()
print("Total number of 'C's in the whole FASTQ dataset:", count_cs(chunks))

Total number of 'C's in the whole FASTQ dataset: 61763321


## Part 5: Using builtin BioNumPy-functions on chunks.

BioNumPy also provides some useful utility-functions for combining results from multiple chunks. One such function is
`bnp.mean` which can take a generator of results computed on chunks and work on the generator as if it only got one big dataset.

For instance, assume we write a function that finds all matches within a chunk:

In [27]:
@bnp.streamable()
def get_matches(chunk, sequence):
    return match_string(chunk.sequence, sequence)

Calling this function on multiple chunks gives us a generator containing the matches for each chunk:

In [28]:
f = bnp.open("ENCFF000RWH_filtered.fastq")
chunks = f.read_chunks()
matches = get_matches(chunks, "CCCTC")
print(type(matches))

<class 'bionumpy.streams.stream.BnpStream'>


Matches is a stream (generator) that will yield the mask for each chunk if we iterate over it.
If we call `bnp.mean` on matches, `bnp.mean` will compute the mean of all the masks in `matches`, combine them correctly,
and return one single number as if it only got **one single mask** for the whole data set:

In [29]:
average_matches_per_base = bnp.mean(matches)

Using this pattern may seem a bit cumbersome until you get used to it, but it enables analysis on larger datasets that can fit into memory.
A common usecase of this pattern is when one wants to compute an average or other statistic over a big data set.

**TASK** Try making a function that simply returns the base qualities for a chunk, and use `bnp.mean` to get the average base qualities for the whole data set.

Compute the average base qualities of the filtered and unfiltered fastq reads (`ENCFF000RWH_filtered.fastq` and `ENCFF000RWH.fastq.gz`).

In [32]:

# Functions to extract base qualities from a chunk
def get_base_qualities(chunk):
    return chunk.quality

# Computes the average base qualities for the whole dataset
def compute_average_base_qualities(filename):
    # Open the FASTQ file
    f = bnp.open(filename)

    # Reads chunks from the file
    chunks = f.read_chunks()

    # Gets base qualities for each chunk
    base_qualities = get_base_qualities(chunks)

    # Computes the average base qualities using bnp.mean
    average_base_qualities = bnp.mean(base_qualities)

    return average_base_qualities

# Computes the average base qualities of the filtered and unfiltered fastq reads
average_base_qualities_filtered = compute_average_base_qualities("ENCFF000RWH_filtered.fastq")
average_base_qualities_unfiltered = compute_average_base_qualities("ENCFF000RWH.fastq.gz")

In [33]:
print("Average base qualities of the filtered fastq reads:", average_base_qualities_filtered)
print("Average base qualities of the unfiltered fastq reads:", average_base_qualities_unfiltered)

Average base qualities of the filtered fastq reads: [31.70837019]
Average base qualities of the unfiltered fastq reads: [30.97913082]


Finding is the average base quality higher in the filtered reads?


In [30]:
@bnp.streamable()
def get_base_qualities(chunk):
    return chunk.quality

f = bnp.open("ENCFF000RWH_filtered.fastq") 
chunks = f.read_chunks()
qualities = get_base_qualities(chunks)
print(bnp.mean(qualities))

[31.70837019]


Important functions used in Bionumpy
1. bnp.as_encoded_array:
Usage: Encodes biological sequences into a NumPy-like array.  Converts DNA sequences into a format that allows for efficient manipulation and analysis using NumPy operations.

2. bnp.open
Usage: Opens a biological data file for reading. Provides an interface to read various bioinformatics file formats (e.g., FASTQ) and process them in chunks.
3. f.read_chunks:
Usage: Reads data from an open file in chunks. Allows for processing large datasets in manageable pieces, improving memory efficiency.

4. bnp.mean: 
Usage: Computes the mean of an array.Calculates the average value, tailored for biological datasets.

5. bnp.streamable():
Usage: Decorates a function to make it streamable, i.e., capable of handling data streams efficiently.

 A typical workflow with BioNumPy typically looks like this:

1. Read one or more data sets with `bnp.open`
2. Use `np`-functions to slice, index or to analyse the data
3. Either write functions that we decorate with `@streamable()` or use builtin-functions to combine results from multiple chunks

Sequence Analysis Algorithms
1. Alignment Algorithms:

Smith-Waterman Algorithm: For local sequence alignment.
Needleman-Wunsch Algorithm: For global sequence alignment.
BLAST (Basic Local Alignment Search Tool): For searching sequence databases.
MAFFT: For multiple sequence alignment.

2. Motif Finding and Pattern Matching:

MEME (Multiple Em for Motif Elicitation): For motif discovery.
FIMO (Find Individual Motif Occurrences): For scanning sequences for motif occurrences.
Regular Expressions: For pattern matching in sequences.

3. Variant Calling and Genotyping:

GATK (Genome Analysis Toolkit): For variant discovery and genotyping.
FreeBayes: For haplotype-based variant detection.
bcftools: For processing and analyzing VCF files.

4. Phylogenetic Analysis:

Neighbor-Joining Algorithm: For constructing phylogenetic trees.
Maximum Likelihood Estimation (MLE): For inferring phylogenies.
Bayesian Inference: For phylogenetic tree estimation.

----->Genomic Data Processing

1. Machine Learning and Statistical Algorithms
Clustering Algorithms:

a. K-means Clustering: For clustering gene expression data.
Hierarchical Clustering: For clustering sequences or expression profiles.

Classification and Regression:

b. Support Vector Machines (SVM): For classification tasks.
Random Forests: For classification and regression.
Neural Networks: For complex pattern recognition in genomic data.
Dimensionality Reduction:

c. Principal Component Analysis (PCA): For reducing dimensionality of expression data.
t-SNE (t-distributed Stochastic Neighbor Embedding): For visualizing high-dimensional data.
UMAP (Uniform Manifold Approximation and Projection): For data visualization and dimensionality reduction.
Evolutionary and Population Genetics
Hardy-Weinberg Equilibrium Testing: For population genetics analysis.
FST Calculation: For measuring population differentiation.
Linkage Disequilibrium Analysis: For studying the non-random association of alleles.

d. Functional Genomics
Gene Set Enrichment Analysis (GSEA): For determining whether a set of genes shows statistically significant differences between two biological states.
DESeq2: For differential gene expression analysis using RNA-seq data.
GO (Gene Ontology) Enrichment Analysis: For functional annotation of gene sets.